PROVA

In [2]:
import os
import random
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
from torchvision.models import resnet50
import torch.nn as nn
import torch.optim as optim

# Estendiamo la classe Dataset per creare una classe personalizzata che utilizza ImageFolder
class CustomDataset(Dataset):
    def __init__(self, root_dir):
        self.image_folder = ImageFolder(root=root_dir)
        # Definizione delle trasformazioni
        self.base_transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
        
        self.rotated_transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.CenterCrop(image_size),
            transforms.RandomRotation(degrees=45),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

    def __getitem__(self, index):
        img, label = self.image_folder[index]
        # Random choice to decide which transform to apply
        if random.random() > 0.5:
            img = self.rotated_transform(img)
        else:
            img = self.base_transform(img)
        return img, label

    def __len__(self):
        return len(self.image_folder)
    
    def number_class(self):
        num_classes = len(os.listdir(self.image_folder.root))
        return num_classes
    
    def name_class(self):
        class_names = sorted(os.listdir(self.image_folder.root))
        return class_names

# Dimensione delle immagini
image_size = 64

# Definizione dei percorsi del dataset
train_data_root = "/home/pituzzu/Scrivania/Tesi Magistrale/Backup_Dataset/Dataset_train_test_val/train"
validation_data_root = "/home/pituzzu/Scrivania/Tesi Magistrale/Backup_Dataset/Dataset_train_test_val/val"
test_data_root = "/home/pituzzu/Scrivania/Tesi Magistrale/Backup_Dataset/Dataset_train_test_val/test"

# Creazione dell'istanza del dataset personalizzato
train_dataset = CustomDataset(root_dir=train_data_root)
validation_dataset = CustomDataset(root_dir=validation_data_root)
test_dataset = CustomDataset(root_dir=test_data_root)

# Definizione del dataloader
batch_size = 128
num_workers = 0 if torch.cuda.is_available() else 2
pin_memory = torch.cuda.is_available()

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                              num_workers=num_workers, pin_memory=pin_memory)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, 
                                   num_workers=num_workers, pin_memory=pin_memory)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, 
                             num_workers=num_workers, pin_memory=pin_memory)

# Info del dataset
print("Info Train:")
print("Numero di classi:", train_dataset.number_class())
print("Nomi delle classi:", train_dataset.name_class())
print("Lunghezza del dataset:", len(train_dataset))

print("Info Validation:")
print("Numero di classi:", validation_dataset.number_class())
print("Nomi delle classi:", validation_dataset.name_class())
print("Lunghezza del dataset:", len(validation_dataset))

print("Info Test:")
print("Numero di classi:", test_dataset.number_class())
print("Nomi delle classi:", test_dataset.name_class())
print("Lunghezza del dataset:", len(test_dataset))

# Definizione del modello
num_classes = train_dataset.number_class()
model = resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Funzione di perdita e ottimizzatore
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Ciclo di addestramento
num_epochs = 200  # Numero di epoche

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_dataloader)}")

    # Validazione
    model.eval()
    validation_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in validation_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    print(f"Validation Loss: {validation_loss / len(validation_dataloader)}, "
          f"Accuracy: {100 * correct / total}%")

# Salvataggio del modello
torch.save(model.state_dict(), "resnet50_finetuned.pth")

# Test del modello
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")


Info Train:
Numero di classi: 8369
Nomi delle classi: ['1', '10', '100', '10000', '10001', '10002', '10003', '10004', '10005', '10006', '10007', '10008', '10009', '1001', '10010', '10012', '10013', '10014', '10015', '10016', '10018', '10019', '1002', '10021', '10023', '10024', '10025', '10026', '10027', '10028', '10029', '1003', '10030', '10031', '10032', '10033', '10034', '10035', '10036', '10037', '10039', '1004', '10041', '10042', '10043', '10044', '10045', '10046', '10047', '10048', '10049', '1005', '10050', '10051', '10052', '10053', '10054', '10055', '10056', '10057', '10058', '10059', '1006', '10060', '10061', '10062', '10064', '10067', '10069', '1007', '10070', '10071', '10072', '10073', '10074', '10075', '10076', '10077', '1008', '10080', '10081', '10082', '10083', '10084', '10085', '10086', '10087', '10089', '1009', '10090', '10091', '10093', '10094', '10095', '10096', '10097', '10098', '10099', '101', '10100', '10101', '10102', '10103', '10104', '10106', '10108', '10109', '1

/home/pituzzu/Scrivania/Tesi Magistrale/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/pituzzu/Scrivania/Tesi Magistrale/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 8.954205592073986
Validation Loss: 8.683123926962576, Accuracy: 0.03798429982273993%
Epoch 2, Loss: 8.340788305304956
Validation Loss: 7.960243435316189, Accuracy: 0.2616696210010973%
Epoch 3, Loss: 7.36114414053746
Validation Loss: 7.226976976599745, Accuracy: 1.3210095382797333%
Epoch 4, Loss: 6.48521397021947
Validation Loss: 6.475655394215738, Accuracy: 3.7731071157255003%
Epoch 5, Loss: 5.769004660435458
Validation Loss: 6.18508840632695, Accuracy: 5.739849750991812%
Epoch 6, Loss: 5.051872003018556
Validation Loss: 5.2740798893795215, Accuracy: 12.923102895247743%
Epoch 7, Loss: 4.309834009039542
Validation Loss: 4.920444168070311, Accuracy: 17.485439351734616%
Epoch 8, Loss: 3.687094953790981
Validation Loss: 4.647196371068237, Accuracy: 22.55845361694944%
Epoch 9, Loss: 3.1314922685000166
Validation Loss: 4.230880730895586, Accuracy: 27.817168903519878%
Epoch 10, Loss: 2.619954842518722
Validation Loss: 4.052866944702723, Accuracy: 31.09648012154976%
Epoch 11, Lo

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


PROVA 23 SETTEMBRE 2024
Resnet50

In [1]:
import os
import random
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
from torchvision.models import resnet50
import torch.nn as nn
import torch.optim as optim

# Classe Early Stopping
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, validation_loss):
        if self.best_loss is None:
            self.best_loss = validation_loss
        elif validation_loss < self.best_loss - self.min_delta:
            self.best_loss = validation_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# Classe CustomDataset per estendere ImageFolder e aggiungere trasformazioni
class CustomDataset(Dataset):
    def __init__(self, root_dir):
        self.image_folder = ImageFolder(root=root_dir)
        # Trasformazione base (senza augmentations)
        self.base_transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
        
        # Trasformazioni con augmentation
        self.augmented_transform = transforms.Compose([
            transforms.Resize(image_size), 
            transforms.RandomHorizontalFlip(p=0.5),  # Capovolge orizzontalmente
            transforms.RandomCrop(image_size, padding=4),  # Ritaglio casuale con padding
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Variazioni di colore
            transforms.RandomRotation(degrees=45),
            transforms.RandomAffine(degrees=10, translate=(0.1, 0.1)),  # Lieve rotazione e traslazione
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

    def __getitem__(self, index):
        img, label = self.image_folder[index]
        # Applica una trasformazione random tra base e augmentation
        if random.random() > 0.5:
            img = self.augmented_transform(img)
        else:
            img = self.base_transform(img)
        return img, label

    def __len__(self):
        return len(self.image_folder)
    
    def number_class(self):
        num_classes = len(os.listdir(self.image_folder.root))
        return num_classes
    
    def name_class(self):
        class_names = sorted(os.listdir(self.image_folder.root))
        return class_names

# Dimensione delle immagini
image_size = 64

# Definizione dei percorsi del dataset
train_data_root = "/home/pituzzu/Scrivania/Tesi Magistrale/Backup_Dataset/Dataset_train_test_val/train"
validation_data_root = "/home/pituzzu/Scrivania/Tesi Magistrale/Backup_Dataset/Dataset_train_test_val/val"
test_data_root = "/home/pituzzu/Scrivania/Tesi Magistrale/Backup_Dataset/Dataset_train_test_val/test"

# Creazione dell'istanza del dataset personalizzato
train_dataset = CustomDataset(root_dir=train_data_root)
validation_dataset = CustomDataset(root_dir=validation_data_root)
test_dataset = CustomDataset(root_dir=test_data_root)

# Definizione del DataLoader
batch_size = 128
num_workers = 0 if torch.cuda.is_available() else 2
pin_memory = torch.cuda.is_available()

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                              num_workers=num_workers, pin_memory=pin_memory)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, 
                                   num_workers=num_workers, pin_memory=pin_memory)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, 
                             num_workers=num_workers, pin_memory=pin_memory)

# Info del dataset
print("Info Train:")
print("Numero di classi:", train_dataset.number_class())
print("Nomi delle classi:", train_dataset.name_class())
print("Lunghezza del dataset:", len(train_dataset))

print("Info Validation:")
print("Numero di classi:", validation_dataset.number_class())
print("Nomi delle classi:", validation_dataset.name_class())
print("Lunghezza del dataset:", len(validation_dataset))

print("Info Test:")
print("Numero di classi:", test_dataset.number_class())
print("Nomi delle classi:", test_dataset.name_class())
print("Lunghezza del dataset:", len(test_dataset))

# Definizione del modello
num_classes = train_dataset.number_class()
model = resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Funzione di perdita e ottimizzatore
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Parametri per l'early stopping
early_stopping = EarlyStopping(patience=10, min_delta=0.01)

# Ciclo di addestramento
num_epochs = 200  # Numero di epoche

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_dataloader)}")

    # Validazione
    model.eval()
    validation_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in validation_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_validation_loss = validation_loss / len(validation_dataloader)
    accuracy = 100 * correct / total
    print(f"Validation Loss: {avg_validation_loss}, Accuracy: {accuracy}%")

    # Early stopping check
    early_stopping(avg_validation_loss)
    if early_stopping.early_stop:
        print("Early stopping")
        break

# Salvataggio del modello
torch.save(model.state_dict(), "resnet50_finetuned.pth")

# Test del modello
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")


Info Train:
Numero di classi: 8369
Nomi delle classi: ['1', '10', '100', '10000', '10001', '10002', '10003', '10004', '10005', '10006', '10007', '10008', '10009', '1001', '10010', '10012', '10013', '10014', '10015', '10016', '10018', '10019', '1002', '10021', '10023', '10024', '10025', '10026', '10027', '10028', '10029', '1003', '10030', '10031', '10032', '10033', '10034', '10035', '10036', '10037', '10039', '1004', '10041', '10042', '10043', '10044', '10045', '10046', '10047', '10048', '10049', '1005', '10050', '10051', '10052', '10053', '10054', '10055', '10056', '10057', '10058', '10059', '1006', '10060', '10061', '10062', '10064', '10067', '10069', '1007', '10070', '10071', '10072', '10073', '10074', '10075', '10076', '10077', '1008', '10080', '10081', '10082', '10083', '10084', '10085', '10086', '10087', '10089', '1009', '10090', '10091', '10093', '10094', '10095', '10096', '10097', '10098', '10099', '101', '10100', '10101', '10102', '10103', '10104', '10106', '10108', '10109', '1

/home/pituzzu/Scrivania/Tesi Magistrale/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/pituzzu/Scrivania/Tesi Magistrale/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 9.022442527751826
Validation Loss: 8.935414145069737, Accuracy: 0.012661433274246644%
Epoch 2, Loss: 8.671379998301342
Validation Loss: 8.400007083851804, Accuracy: 0.12239385498438424%
Epoch 3, Loss: 7.7832859145536695
Validation Loss: 7.578241860994729, Accuracy: 0.531780197518359%
Epoch 4, Loss: 7.0164581251703515
Validation Loss: 6.918120725180513, Accuracy: 1.6966320587490504%
Epoch 5, Loss: 6.423124546980738
Validation Loss: 6.55032665242431, Accuracy: 3.1653583185616614%
Epoch 6, Loss: 5.837494658465362
Validation Loss: 6.139393560348019, Accuracy: 5.6469992403140035%
Epoch 7, Loss: 5.272327306082899
Validation Loss: 5.536789647994503, Accuracy: 10.479446273318139%
Epoch 8, Loss: 4.729977597143981
Validation Loss: 5.173074468489616, Accuracy: 13.822064657719254%
Epoch 9, Loss: 4.194857144874943
Validation Loss: 4.866345677324521, Accuracy: 17.637376551025575%
Epoch 10, Loss: 3.7637130928598657
Validation Loss: 4.520399837083714, Accuracy: 21.97180720857601%
Epoch 

PROVA 23 SETTEMBRE 2024
Resnet18

In [2]:
import os
import random
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
from torchvision.models import resnet18  # Cambiato da resnet50 a resnet18
import torch.nn as nn
import torch.optim as optim

# Classe Early Stopping
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, validation_loss):
        if self.best_loss is None:
            self.best_loss = validation_loss
        elif validation_loss < self.best_loss - self.min_delta:
            self.best_loss = validation_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

# Classe CustomDataset per estendere ImageFolder e aggiungere trasformazioni
class CustomDataset(Dataset):
    def __init__(self, root_dir):
        self.image_folder = ImageFolder(root=root_dir)
        # Trasformazione base (senza augmentations)
        self.base_transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
        
        # Trasformazioni con augmentation
        self.augmented_transform = transforms.Compose([
            transforms.Resize(image_size), 
            transforms.RandomHorizontalFlip(p=0.5),  # Capovolge orizzontalmente
            transforms.RandomCrop(image_size, padding=4),  # Ritaglio casuale con padding
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Variazioni di colore
            transforms.RandomRotation(degrees=45),
            transforms.RandomAffine(degrees=10, translate=(0.1, 0.1)),  # Lieve rotazione e traslazione
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

    def __getitem__(self, index):
        img, label = self.image_folder[index]
        # Applica una trasformazione random tra base e augmentation
        if random.random() > 0.5:
            img = self.augmented_transform(img)
        else:
            img = self.base_transform(img)
        return img, label

    def __len__(self):
        return len(self.image_folder)
    
    def number_class(self):
        num_classes = len(os.listdir(self.image_folder.root))
        return num_classes
    
    def name_class(self):
        class_names = sorted(os.listdir(self.image_folder.root))
        return class_names

# Dimensione delle immagini
image_size = 64

# Definizione dei percorsi del dataset
train_data_root = "/home/pituzzu/Scrivania/Tesi Magistrale/Backup_Dataset/Dataset_train_test_val/train"
validation_data_root = "/home/pituzzu/Scrivania/Tesi Magistrale/Backup_Dataset/Dataset_train_test_val/val"
test_data_root = "/home/pituzzu/Scrivania/Tesi Magistrale/Backup_Dataset/Dataset_train_test_val/test"

# Creazione dell'istanza del dataset personalizzato
train_dataset = CustomDataset(root_dir=train_data_root)
validation_dataset = CustomDataset(root_dir=validation_data_root)
test_dataset = CustomDataset(root_dir=test_data_root)

# Definizione del DataLoader
batch_size = 128
num_workers = 0 if torch.cuda.is_available() else 2
pin_memory = torch.cuda.is_available()

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                              num_workers=num_workers, pin_memory=pin_memory)
validation_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, 
                                   num_workers=num_workers, pin_memory=pin_memory)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, 
                             num_workers=num_workers, pin_memory=pin_memory)

# Info del dataset
print("Info Train:")
print("Numero di classi:", train_dataset.number_class())
print("Nomi delle classi:", train_dataset.name_class())
print("Lunghezza del dataset:", len(train_dataset))

print("Info Validation:")
print("Numero di classi:", validation_dataset.number_class())
print("Nomi delle classi:", validation_dataset.name_class())
print("Lunghezza del dataset:", len(validation_dataset))

print("Info Test:")
print("Numero di classi:", test_dataset.number_class())
print("Nomi delle classi:", test_dataset.name_class())
print("Lunghezza del dataset:", len(test_dataset))

# Definizione del modello
num_classes = train_dataset.number_class()
model = resnet18(pretrained=True)  # Usa resnet18 invece di resnet50
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Adatta l'ultimo strato al numero di classi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Funzione di perdita e ottimizzatore
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Parametri per l'early stopping
early_stopping = EarlyStopping(patience=10, min_delta=0.01)

# Ciclo di addestramento
num_epochs = 200  # Numero di epoche

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_dataloader)}")

    # Validazione
    model.eval()
    validation_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in validation_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_validation_loss = validation_loss / len(validation_dataloader)
    accuracy = 100 * correct / total
    print(f"Validation Loss: {avg_validation_loss}, Accuracy: {accuracy}%")

    # Early stopping check
    early_stopping(avg_validation_loss)
    if early_stopping.early_stop:
        print("Early stopping")
        break

# Salvataggio del modello
torch.save(model.state_dict(), "resnet18_finetuned.pth")  # Cambiato il nome del file

# Test del modello
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")


Info Train:
Numero di classi: 8369
Nomi delle classi: ['1', '10', '100', '10000', '10001', '10002', '10003', '10004', '10005', '10006', '10007', '10008', '10009', '1001', '10010', '10012', '10013', '10014', '10015', '10016', '10018', '10019', '1002', '10021', '10023', '10024', '10025', '10026', '10027', '10028', '10029', '1003', '10030', '10031', '10032', '10033', '10034', '10035', '10036', '10037', '10039', '1004', '10041', '10042', '10043', '10044', '10045', '10046', '10047', '10048', '10049', '1005', '10050', '10051', '10052', '10053', '10054', '10055', '10056', '10057', '10058', '10059', '1006', '10060', '10061', '10062', '10064', '10067', '10069', '1007', '10070', '10071', '10072', '10073', '10074', '10075', '10076', '10077', '1008', '10080', '10081', '10082', '10083', '10084', '10085', '10086', '10087', '10089', '1009', '10090', '10091', '10093', '10094', '10095', '10096', '10097', '10098', '10099', '101', '10100', '10101', '10102', '10103', '10104', '10106', '10108', '10109', '1

/home/pituzzu/Scrivania/Tesi Magistrale/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/pituzzu/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [02:36<00:00, 300kB/s]


Epoch 1, Loss: 9.17547499674249
Validation Loss: 9.014515282005393, Accuracy: 0.021102388790411073%
Epoch 2, Loss: 8.720121669609542
Validation Loss: 8.261361563077537, Accuracy: 0.13083481050054865%
Epoch 3, Loss: 7.761753708872963
Validation Loss: 7.468021977332331, Accuracy: 0.5571030640668524%
Epoch 4, Loss: 7.0398393128585175
Validation Loss: 6.9494296914787705, Accuracy: 1.730395880813708%
Epoch 5, Loss: 6.403477829305371
Validation Loss: 6.5929817999562905, Accuracy: 3.0809487634000168%
Epoch 6, Loss: 5.829522215940645
Validation Loss: 6.000847126847955, Accuracy: 6.681016291044147%
Epoch 7, Loss: 5.3514682759392
Validation Loss: 5.708862225214641, Accuracy: 8.867223769730733%
Epoch 8, Loss: 4.890653508791772
Validation Loss: 5.45350762721031, Accuracy: 11.479699501983625%
Epoch 9, Loss: 4.4775300281531045
Validation Loss: 5.121023642119541, Accuracy: 15.98716974761543%
Epoch 10, Loss: 4.094043041194104
Validation Loss: 5.004942345362838, Accuracy: 18.59964547986832%
Epoch 11, L